In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors


In [2]:
data = pd.read_csv('C:\project_mlops\datasets\data_final.csv')

In [35]:
data[['title','genres']]

,title,genres
0,Minions,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '..."
1,Wonder Woman,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."
2,Beauty and the Beast,"[{'id': 10751, 'name': 'Family'}, {'id': 14, '..."
3,Baby Driver,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam..."
4,Big Hero 6,"[{'id': 12, 'name': 'Adventure'}, {'id': 10751..."
...,...,...
45341,At the River I Stand,[]
45342,Narrien Illat,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
45343,The Winner,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n..."
45344,Raakh,[]


In [4]:
import ast
import json

def convertir_a_dict(valor):
    datos_list = ast.literal_eval(valor)
    datos_str_list = [json.dumps(d) for d in datos_list]
    datos_dict_list = [json.loads(d) for d in datos_str_list]
    return datos_dict_list

# Aplicar la función a toda la columna 'cast' agregando una nueva columna al DF llamada 'cast_dict'
data['genres'] = data['genres'].apply(convertir_a_dict)

In [30]:
data['genres1'] = data['genres'].apply(lambda x: ' '.join([d['name'] for d in x]) if isinstance(x, list) else None)

In [31]:
data['genres1']

0               Family Animation Adventure Comedy
1                        Action Adventure Fantasy
2                          Family Fantasy Romance
3                                    Action Crime
4        Adventure Family Animation Action Comedy
                           ...                   
45341                                            
45342                          Comedy Drama Music
45343                       Drama Romance Foreign
45344                                            
45345                                      Comedy
Name: genres1, Length: 45346, dtype: object

In [7]:

import json
import ast

# primero le hacemos un tratamiento al texto con el modulo ast para pasar el texto a un formato mas entendible por python 
def transform_collection_name(value):
    if pd.isnull(value): # si en la fila existe algún elemento en Nan lo deja igual 
        return value
    collection_dict = ast.literal_eval(value)
    value = value.replace("'", '"')  # Reemplazar comillas simples por comillas dobles
    # Obtener el valor asociado a la clave 'name'
    name_value = collection_dict['name']
    # Para los valores en 'name' con algunos caracteres extraños se les intercambia por espacio
    # Reemplazar comas por una cadena vacía en name_value
    name_value = name_value.replace(',', '').replace("!", '').replace("'", '').replace("*", '').replace("-", '')  
    # Actualizar el valor 
    collection_dict['name'] = name_value

    # Convertimos lo que parece un diccionario de vuelta a texto
    updated_value = str(collection_dict)

    return updated_value

# Esta función nos permite evaluar el texto de la fila y mediante el modulo de json y luego poder extraer cada valor y crear el diccionario
def extract_collection_info(row):
    try:
        if pd.isnull(row): # si en la fila existe algún elemento en Nan lo deja igual
            return row
        row = transform_collection_name(row)
        # Reemplazar comillas simples por comillas dobles
        row = row.replace("'", '"')
        collection_data = json.loads(row)
        if not isinstance(collection_data, dict):
            raise ValueError("Invalid JSON format")

        id = collection_data.get('id')
        name = collection_data.get('name')
        poster_path = collection_data.get('poster_path')
        backdrop_path = collection_data.get('backdrop_path')
        
        # Manejar el valor None en backdrop_path y poster_path
        if backdrop_path == 'None':
            backdrop_path = None
        if poster_path == 'None':
            poster_path = None
        
        return {'id': id,
                'name': name,
                'poster_path': poster_path,
                'backdrop_path': backdrop_path}
    except (json.JSONDecodeError, ValueError) as e:
        return {'error': str(e)}

In [8]:
#data['belongs_to_collection'] = data['belongs_to_collection'].apply(extract_collection_info)

In [11]:
data['belongs_to_collection'][0]

{'id': 86066,
 'name': 'Despicable Me Collection',
 'poster_path': '/xIXhIlZDRmSSfNbpN7kBCm5hg39.jpg',
 'backdrop_path': '/15IZl405E664QDVxpFJBl7TtLmw.jpg'}

In [13]:
valores_name = data['belongs_to_collection'].apply(lambda x: x.get('name') if not pd.isna(x) else None)

In [16]:
data['belongs_to_collection'] = valores_name

In [38]:
data['genres1']

0               Family Animation Adventure Comedy
1                        Action Adventure Fantasy
2                          Family Fantasy Romance
3                                    Action Crime
4        Adventure Family Animation Action Comedy
                           ...                   
45341                                            
45342                          Comedy Drama Music
45343                       Drama Romance Foreign
45344                                            
45345                                      Comedy
Name: genres1, Length: 45346, dtype: object

In [41]:
data_ml = data[['title','overview','belongs_to_collection','genres1']]

In [42]:
data_ml

,title,overview,belongs_to_collection,genres1
0,Minions,"Minions Stuart, Kevin and Bob are recruited by...",Despicable Me Collection,Family Animation Adventure Comedy
1,Wonder Woman,An Amazon princess comes to the world of Man t...,Wonder Woman Collection,Action Adventure Fantasy
2,Beauty and the Beast,A live-action adaptation of Disney's version o...,None,Family Fantasy Romance
3,Baby Driver,After being coerced into working for a crime b...,None,Action Crime
4,Big Hero 6,The special bond that develops between plus-si...,None,Adventure Family Animation Action Comedy
...,...,...,...,...
45341,At the River I Stand,This moving documentary recounts the two month...,None,
45342,Narrien Illat,The ups and downs of the popular music world a...,None,Comedy Drama Music
45343,The Winner,Tou former boxers meet in the ring again after...,None,Drama Romance Foreign
45344,Raakh,After witnessing the public humiliation &amp; ...,None,


In [43]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\felix\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\felix\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\felix\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [44]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

nltk.download('stopwords')

# Convertir todas las columnas de texto a minúsculas
data_ml['overview'] = data_ml['overview'].astype(str).str.lower()
data_ml['title'] = data_ml['title'].astype(str).str.lower()
data_ml['belongs_to_collection'] = data_ml['belongs_to_collection'].astype(str).str.lower()
data_ml['genres1'] = data_ml['genres1'].astype(str).str.lower()

# Eliminar caracteres no deseados en todas las columnas
data_ml['overview'] = data_ml['overview'].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))
data_ml['title'] = data_ml['title'].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))
data_ml['belongs_to_collection'] = data_ml['belongs_to_collection'].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))
data_ml['genres1'] = data_ml['genres1'].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))

# Eliminar palabras vacías (stop words) en todas las columnas
stop_words = set(stopwords.words('english'))
data_ml['overview'] = data_ml['overview'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word.lower() not in stop_words]))
data_ml['title'] = data_ml['title'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word.lower() not in stop_words]))
data_ml['belongs_to_collection'] = data_ml['belongs_to_collection'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word.lower() not in stop_words]))
data_ml['genres1'] = data_ml['genres1'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word.lower() not in stop_words]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\felix\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\felix\AppData\Local\Temp\ipykernel_24292\779913855.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ml['overview'] = data_ml['overview'].astype(str).str.lower()
C:\Users\felix\AppData\Local\Temp\ipykernel_24292\779913855.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ml['title'] = data_ml['title'].astype(str).str.lower()
C

In [45]:
data_ml

,title,overview,belongs_to_collection,genres1
0,minions,minions stuart kevin bob recruited scarlet ove...,despicable collection,family animation adventure comedy
1,wonder woman,amazon princess comes world man become greates...,wonder woman collection,action adventure fantasy
2,beauty beast,live action adaptation disney version classic ...,none,family fantasy romance
3,baby driver,coerced working crime boss young getaway drive...,none,action crime
4,big hero,special bond develops plus sized inflatable ro...,none,adventure family animation action comedy
...,...,...,...,...
45341,river stand,moving documentary recounts two months leading...,none,
45342,narrien illat,ups downs popular music world revealed singer,none,comedy drama music
45343,winner,tou former boxers meet ring years run old matc...,none,drama romance foreign
45344,raakh,witnessing public humiliation amp abuse female...,none,


In [63]:
#data_ml.to_csv('data_ml.csv', index=False)

In [2]:
data_ml = pd.read_csv('C:\project_mlops\datasets\data_ml.csv')

In [3]:
data_ml['combined_text'] = data_ml['overview'] + ' ' + data_ml['title'] + ' ' + data_ml['belongs_to_collection'] + ' ' + data_ml['genres1']


In [23]:
data_ml = data_ml.dropna(subset=['combined_text'])
data_ml = data_ml.reset_index(drop=True)

In [31]:
data_ML = data_ml[['title','combined_text']]

In [32]:
data_ML.to_csv('data_ML.csv', index=False)

In [2]:
import pandas as pd

In [3]:
data_ml = pd.read_csv('C:\project_mlops\datasets\data_ML.csv')

In [6]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

vectorizer = TfidfVectorizer(stop_words='english')
vectorized_data = vectorizer.fit_transform(data_ml['combined_text'])

In [7]:
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(vectorized_data)

NearestNeighbors(algorithm='brute', metric='cosine')

In [23]:
# Crear y ajustar el modelo KNN fuera de la función
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(vectorized_data)
import re

# Compilar la expresión regular fuera de la función
regex = re.compile(r'[^a-zA-Z]')
def get_recomendations(title, modelo=knn_model, regex=regex):
    
    # Preprocesamiento del título
    processed_title = re.sub(r'[^a-zA-Z]', ' ', title.lower())
    
    # Obtener índice de la película de consulta
    index = data_ml[data_ml['title'].str.lower() == processed_title].index[0]
    
    # Numero de recomendaciones
    num_recomen=5
    
    # Obtener recomendaciones basadas en el índice de consulta
    _, indices = knn_model.kneighbors(vectorized_data[index], n_neighbors=num_recomen+1)
    
    # Obtener índices de las películas recomendadas
    index_title = indices.flatten()[1:]
    
    # Devolver una estructura de diccionario con los títulos recomendados
    result = {'lista recomendada': [title.title() for title in data_ml['title'].iloc[index_title].tolist()]}

    return result

In [25]:
titulo = input('Ingrese un título: ')
recomendacion = get_recomendations(titulo)
print(recomendacion)

{'lista recomendada': ['Shrek', 'Shrek Third', 'Shrek Forever', 'Shrek Halls', 'Scared Shrekless']}
